# SEIRS+ Graph Model Demo

**This notebook provides a demonstration of the core functionality of the SEIRS+ Graph Model and offers a sandbox for easily changing simulation parameters and scenarios.** 
For a more thorough walkthrough of the model and use of this package, refer to the README and tutorial notebook.

### Installing and Importing the model code
All of the code needed to run the model is imported from the ```models``` module of this package.

#### Install the package using ```pip```
The package can be installed on your machine by entering this in the command line:

```sudo pip install SEIRS-network-model```

Then, the ```models``` module can be imported into your scripts as shown here:

In [2]:
from SEIRS-network-model.models import SEIRSGraphModel # Import the SEIRS+ graph model.
from SEIRS-network-model.models import custom_exponential_graph
import networkx

SyntaxError: invalid syntax (<ipython-input-2-a600228fead8>, line 1)

#### *Alternatively, manually copy the code to your machine*
*You can use the model code without installing a package by copying the ```models.py``` module file to a directory on your machine. In this case, the easiest way to use the module is to place your scripts in the same directory as the module, and import the module as shown here:*
```python
from models import SEIRSGraphModel # Import the SEIRS+ graph model.
from models import custom_exponential_graph
```

### Generating interaction networks

This model simulates SEIRS epidemic dynamics for populations with a structured interaction network (as opposed to standard deterministic SIR/SEIR/SEIRS models, which assume uniform mixing of the population). As such, a graph specifying the interaction network for the population must be specified, where each node represents an individual in the population and edges connect individuals who have regular interactions.

The interaction network can be specified by a ```networkx``` Graph object or a 2D numpy array representing the adjacency matrix, either of which can be defined and generated by any method.

*Here, we use a ```custom_exponential_graph()``` generation function included in this package, which generates power-law graphs that have degree distributions with two exponential tails. For more information on this custom graph type and its generation, see the README.*

**_Note: Simulations of networks with more than **

In [ ]:
numNodes = 10000
baseGraph    = networkx.barabasi_albert_graph(n=numNodes, m=9)
# Baseline normal interactions:
G_normal     = custom_exponential_graph(baseGraph, scale=100)

Epidemic scenarios of interest often involve interaction networks that change in time. Multiple interaction networks can be defined and used at different times in the model simulation, as will be shown below.

*Here we generate a graph representing interactions during corresponding to Social Distancing, where each individual drops some portion of their normal interactions with others. Again, we use the ```custom_exponential_graph()``` to generate this graph; for more information, see the README.*

In [ ]:
# Social distancing interactions:
G_distancing = custom_exponential_graph(baseGraph, scale=10)

This SEIRS+ model features dynamics corresponding to testing individuals for the disease and moving individuals with detected infection into a state where their rate of recovery, mortality, etc may be different. In addition, given that this model considers individuals in an interaction network, a separate graph defining the interactions for individuals with detected cases can be specified.

*Here we generate a graph representing the interactions that individuals have when they test positive for the disease. In this case, a significant portion of each individual's normal interaction edges are removed from the graph, as if the individual is quarantined upon detection of infection. Again, we use the ```custom_exponential_graph()``` to generate this graph; for more information, see the README.*

For more information on how testing, contact tracing, and detected cases are handled in this model, see the README.

In [ ]:
# Quarantine interactions:
G_quarantine = custom_exponential_graph(baseGraph, scale=5)

### Initializing the model parameters
All model parameter values, including the normal and quarantine interaction networks, are set in the call to the ```SEIRSGraphModel``` constructor. The normal interaction network ```G``` and the basic SEIR parameters ```beta```, ```sigma```, and ```gamma``` are the only required arguments. All other arguments represent parameters for optional extended model dynamics; these optional parameters take default values that turn off their corresponding dynamics when not provided in the constructor. For clarity and ease of customization in this notebook, all available model parameters are listed below. 

For more information on parameter meanings, see the README and tutorial notebook.

*The parameter values shown correspond to rough estimates of parameter values for the COVID-19 epidemic.*

In [ ]:
model = SEIRSGraphModel(G       =G_normal, 
                        beta    =0.155, 
                        sigma   =1/5.2, 
                        gamma   =1/12.39, 
                        mu_I    =0.0004,
                        mu_0    =0, 
                        nu      =0, 
                        xi      =0,
                        p       =0.5,
                        Q       =G_quarantine, 
                        beta_D  =0.155, 
                        sigma_D =1/5.2, 
                        gamma_D =1/12.39, 
                        mu_D    =0.0004,
                        theta_E =0, 
                        theta_I =0, 
                        phi_E   =0, 
                        phi_I   =0, 
                        psi_E   =1.0, 
                        psi_I   =1.0,
                        q       =0.5,
                        initI   =numNodes/100, 
                        initE   =0, 
                        initD_E =0, 
                        initD_I =0, 
                        initR   =0, 
                        initF   =0)

### Checkpoints
Model parameters can be easily changed during a simulation run using checkpoints. A dictionary holds a list of checkpoint times (```checkpoints['t']```) and lists of new values to assign to various model parameters at each checkpoint time. Only model parameters included in the checkpoints dictionary have their values updated at the checkpoint times, all other parameters keep their pre-existing values.

*The checkpoints shown here correspond to starting social distancing and testing at time ```t=20``` (the graph ```G``` is updated to ```G_distancing``` and locality parameter ```p``` is decreased to ```0.1```; testing params ```theta_E```, ```theta_I```, ```phi```, and ```phi_I``` are set to non-zero values) and then stopping social distancing at time ```t=100``` (```G``` and ```p``` changed back to their "normal" values; testing params remain non-zero).*

In [ ]:
checkpoints = {'t':       [20, 100], 
               'G':       [G_distancing, G_normal], 
               'p':       [0.1, 0.5], 
               'theta_E': [0.02, 0.02], 
               'theta_I': [0.02, 0.02], 
               'phi_E':   [0.2, 0.2], 
               'phi_I':   [0.2, 0.2]}

### Running the simulation

In [ ]:
model.run(T=300, checkpoints=checkpoints)

### Visualizing the results
The ```SEIRSGraphModel``` class has a ```plot()``` convenience function for plotting simulation results on a matplotlib axis. This function generates a line plot of the frequency of each model state in the population by default, but there are many optional arguments that can be used to customize the plot.

The ```SEIRSGraphModel``` class also has convenience functions for generating a full figure out of model simulation results (optionaly arguments can be provided to customize the plots generated by these functions). 
- ```figure_basic()``` calls the ```plot()``` function with default parameters to generate a line plot of the frequency of each state in the population.
- ```figure_infections()``` calls the ```plot()``` function with default parameters to generate a stacked area plot of the frequency of only the infection states ($E$, $I$, $D_E$, $D_I$) in the population.

For more information on the built-in plotting functions, see the README and tutorial notebook.

In [ ]:
model.figure_infections(vlines=checkpoints['t'], ylim=0.15)